In [ ]:
!pip install -q pandas seaborn numpy matplotlib langdetect

In [1]:
import requests
import pandas as pd

url = 'https://hub.snapshot.org/graphql'

headers = {
    'Content-Type': 'application/json'
}

payload = {
    'query': ""
}

def request(query:str):
    payload['query'] = query
    response = requests.post(url, headers=headers, json=payload)
    return response

In [2]:
from util.api import DAO, TOP20
dao = DAO()

# Space

In [97]:
result = []
for i in range(0, 90000, 1000):
    query = dao.fetch_spaces(i)
    response = request(query)
    result += response.json()["data"]["spaces"]
    print(i, len(result))

df = pd.DataFrame(result)
df.to_excel('spaces.xlsx', index=False, engine='xlsxwriter')

0 1000
1000 2000
2000 3000
3000 4000
4000 5000
5000 6000
6000 7000
7000 8000
8000 9000
9000 10000
10000 11000
11000 12000
12000 13000
13000 14000
14000 15000
15000 16000
16000 17000
17000 18000
18000 19000
19000 20000
20000 21000
21000 22000
22000 23000
23000 24000
24000 25000
25000 26000
26000 27000
27000 28000
28000 29000
29000 30000
30000 31000
31000 32000
32000 33000
33000 34000
34000 35000
35000 36000
36000 37000
37000 38000
38000 39000
39000 40000
40000 41000
41000 42000
42000 43000
43000 44000
44000 45000
45000 46000
46000 47000
47000 48000
48000 49000
49000 50000
50000 51000
51000 52000
52000 53000
53000 54000
54000 55000
55000 56000
56000 57000
57000 58000
58000 59000
59000 60000
60000 61000
61000 62000
62000 63000
63000 64000
64000 65000
65000 66000
66000 67000
67000 68000
68000 69000
69000 70000
70000 71000
71000 72000
72000 73000
73000 74000
74000 75000
75000 76000
76000 77000
77000 78000
78000 79000
79000 80000
80000 81000
81000 82000
82000 83000
83000 84000
84000 85000
85

In [98]:
print(df.columns)

Index(['created', 'id', 'name', 'skin', 'rank', 'about', 'terms', 'email',
       'turbo', 'boost', 'avatar', 'github', 'symbol', 'domain', 'admins',
       'voting', 'parent', 'private', 'website', 'twitter', 'network',
       'members', 'filters', 'plugins', 'flagged', 'location', 'children',
       'template', 'verified', 'coingecko', 'strategies', 'moderators',
       'categories', 'validation', 'treasuries', 'votesCount', 'guidelines',
       'hibernated', '__typename', 'votesCount7d', 'voteValidation',
       'proposalsCount', 'followersCount', 'activeProposals',
       'delegationPortal', 'proposalsCount7d', 'followersCount7d'],
      dtype='object')


In [99]:
print(df.shape)

print(df[df["votesCount"]>1000].shape)

print(df[df["proposalsCount"]>100].shape)

print(df[df["followersCount"]>1000].shape)

print(df[df["followersCount"]>1000].loc[df["proposalsCount"]>20].shape)

(87795, 47)
(775, 47)
(318, 47)
(288, 47)
(171, 47)


In [100]:
for t in TOP20:
    if not df[df["id"]== t].empty:
        print(t)
        print(df[df["id"]== t].shape)
        print(df[df["id"]== t][["proposalsCount", "followersCount", "votesCount"]])

0xgov.eth
(1, 47)
      proposalsCount  followersCount  votesCount
1266              27            2023        2098
1inch.eth
(1, 47)
      proposalsCount  followersCount  votesCount
1027              61           13428       11914
aave.eth
(1, 47)
      proposalsCount  followersCount  votesCount
1087             663          148831     3128667
aavegotchi.eth
(1, 47)
    proposalsCount  followersCount  votesCount
48             509           85774      409593
arbitrumfoundation.eth
(1, 47)
       proposalsCount  followersCount  votesCount
13714             295          313277     5285417
badgerdao.eth
(1, 47)
    proposalsCount  followersCount  votesCount
44             117           10526       72204
balancer.eth
(1, 47)
      proposalsCount  followersCount  votesCount
1505             810           34052      123061
comp-vote.eth
(1, 47)
     proposalsCount  followersCount  votesCount
388               8            7154        2304
cvx.eth
(1, 47)
      proposalsCount  followersCount

# Proposal

In [106]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_excel('spaces.xlsx')
df.shape

(87795, 47)

In [107]:
df_cut = df[df["proposalsCount"]>20].loc[df["followersCount"]>100]
print(df_cut["votesCount"].quantile(0.1))
df_cut = df_cut[df_cut["votesCount"]>df_cut["votesCount"].quantile(0.1)]
df_cut.to_excel('spaces_cut.xlsx', index=False, engine='xlsxwriter')
df_cut.shape

188.60000000000002


(552, 47)

In [109]:
result = []
spaces = df_cut['id'].tolist()

failed_space = []
for i, space in enumerate(spaces):
    print(i, space)
    idx = 0
    while True:
        query = dao.fecth_proposals(space, idx)
        response = request(query).json()
        if 'error' in response.keys():
            print(response)
            failed_space.append(space)
            continue
        response = response["data"]["proposals"]
        if response:
            result += response
            print(len(response), len(result))
            idx += 1000
        else:
            break

0 fabien.eth
418 418
1 idlefinance.eth
119 537
2 dodobird.eth
23 560
3 decentralgames.eth
267 827
4 badgerdao.eth
117 944
5 yam.eth
215 1159
6 aavegotchi.eth
514 1673
7 baovotes.eth
81 1754
8 unidexapp.eth
65 1819
9 xdaistake.eth
257 2076
10 truefigov.eth
78 2154
11 frax.eth
461 2615
12 nftx.eth
81 2696
13 ndx.eth
51 2747
14 friendswithbenefits.eth
79 2826
15 vote-perp.eth
38 2864
16 barnbridge.eth
32 2896
17 cream-finance.eth
112 3008
18 stakedao.eth
109 3117
19 defigeek.eth
83 3200
20 community.nexusmutual.eth
63 3263
21 vote.airswap.eth
112 3375
22 metafactory.eth
64 3439
23 lido-snapshot.eth
318 3757
24 stabilize-governance.eth
37 3794
25 opiumprotocol.eth
86 3880
26 gov.dhedge.eth
89 3969
27 diadao.eth
99 4068
28 premia.eth
45 4113
29 alchemistcoin.eth
21 4134
30 snapshot.floatprotocol.eth
61 4195
31 gnosis.eth
196 4391
32 alpacafinance.eth
48 4439
33 tokenlon.eth
45 4484
34 alchemixstakers.eth
130 4614
35 tracer.eth
46 4660
36 snxgov.eth
853 5513
37 insuretoken.eth
111 5624
38 qu

In [110]:
from datetime import datetime

df_proposals = pd.DataFrame(result) # 'strategies', 'validation'

# data cleansing: {space: {id}} -> {space_id}
df_proposals["space_id"] = df_proposals["space"].apply(lambda x: x["id"])
del df_proposals["space"]

# data cleansing: 1722174165 -> datatime(2024-07-28 22:42:??)
for v in ['start', 'end', 'created']:
    df_proposals[v] = df_proposals[v].apply(datetime.fromtimestamp)

print(df_proposals.shape, df_proposals.columns)

(78979, 33) Index(['id', 'end', 'app', 'ipfs', 'type', 'body', 'link', 'title', 'start',
       'state', 'votes', 'author', 'symbol', 'quorum', 'scores', 'created',
       'updated', 'network', 'plugins', 'choices', 'privacy', 'flagged',
       'snapshot', 'strategies', 'validation', 'discussion', 'quorumType',
       '__typename', 'scores_state', 'scores_total', 'scores_updated',
       'scores_by_strategy', 'space_id'],
      dtype='object')


In [111]:
df_proposals.head()

,id,end,app,ipfs,type,body,link,title,start,state,...,strategies,validation,discussion,quorumType,__typename,scores_state,scores_total,scores_updated,scores_by_strategy,space_id
0,0xd77a8cf8575bdc4bd22f008c24e44b4f130d600b11ee...,2024-08-10 05:43:01,snapshot-v-2,bafkreif5juzrnhpw3ne6sh3vz4z7pl4iagi7tj5ubm4c2...,basic,,https://snapshot.org/#/fabien.eth/proposal/0xd...,Test proposal,2024-08-10 04:43:01,closed,...,"[{'network': '1', 'params': {'symbol': 'ETH'}}]",{'params': {}},,default,Proposal,final,0.284466,1723236196,"[[0], [0.2844664907898591], [0]]",fabien.eth
1,0x72f473f257c9730ae959a0f268580a17c514edcecdc2...,2024-08-08 20:17:37,snapshot-v-2,bafkreicirzyfobjz4uqpdx3risae3dng33l7z4joupnmr...,basic,,https://snapshot.org/#/fabien.eth/proposal/0x7...,Test proposal with execution,2024-08-08 19:17:37,closed,...,"[{'network': '1', 'params': {'symbol': 'ETH'}}]",{'params': {}},,default,Proposal,final,0.284466,1723115875,"[[0.2844664907898591], [0], [0]]",fabien.eth
2,0xe887785955da0967828c15c34922287b051d00c237b0...,2024-08-07 19:23:15,snapshot-v-2,bafkreiercujbijibwxqlovv7aucffi56nm3kmbrrazguf...,basic,,https://snapshot.org/#/fabien.eth/proposal/0xe...,Test proposal,2024-08-07 18:23:15,closed,...,"[{'network': '1', 'params': {'symbol': 'ETH'}}]",{'params': {}},,default,Proposal,final,0,1723026213,"[[0], [0], [0]]",fabien.eth
3,0xc4697cca0148a2178b6687059fd846b60889beb3e526...,2024-08-05 23:25:15,snapshot-v-2,bafkreicu5a7ehvl4b53ob6uytj4akfm7o43ictp35n4bo...,basic,,https://snapshot.org/#/fabien.eth/proposal/0xc...,Test proposal 2,2024-08-05 22:25:15,closed,...,"[{'network': '1', 'params': {'symbol': 'ETH'}}]",{'params': {}},,default,Proposal,final,0,1722867925,"[[0], [0], [0]]",fabien.eth
4,0xeadee7de89dfa66e78ad94594b0c8f5644f5163e477f...,2024-08-05 23:22:38,snapshot-v-2,bafkreibl7zzc6ocgw2pbnpwl5ewxbz62zlzcekrn76mgi...,basic,,https://snapshot.org/#/fabien.eth/proposal/0xe...,Test proposal,2024-08-05 22:22:38,closed,...,"[{'network': '1', 'params': {'symbol': 'ETH'}}]",{'params': {}},,default,Proposal,final,0.285126,1722867773,"[[0.28512600073476874], [0], [0]]",fabien.eth


In [112]:
# df_proposals의 start에서 2024년 이후의 데이터가 존재하는 space만 추출

temp = df_proposals[df_proposals['start'] >= pd.to_datetime('2024-01-01')]
spaces = temp["space_id"].unique()
print(len(spaces))

df_proposals = df_proposals[df_proposals["space_id"].isin(spaces)]
df_proposals.shape

297


(46380, 33)

In [113]:
# state가 closed인 데이터만 추출
df_proposals = df_proposals[df_proposals['state'] == 'closed']
df_proposals.shape, df_proposals["space_id"].nunique()

((46144, 33), 296)

In [114]:
# title, body 중복 제거
# df_proposals = df_proposals.sort_values(by=["created"])
df_proposals = df_proposals.drop_duplicates(subset=['title', 'body'], keep="last")
print("Drop duplicated in title & body:", df_proposals.shape)

# title, body가 NaN인 데이터 제거
df_proposals = df_proposals.dropna(subset=['title', 'body'])

# body가 150자 이상인 데이터만 추출
df_proposals = df_proposals[df_proposals['body'].apply(lambda x: len(str(x)) >= 150)]
print("Over 150 body length:", df_proposals.shape)
df_proposals["space_id"].nunique()

Drop duplicated in title & body: (42578, 33)
Over 150 body length: (28728, 33)


266

In [115]:
df_proposals['votes'].describe()

count     28728.000000
mean       1079.387601
std       14992.240949
min           0.000000
25%           9.000000
50%          31.000000
75%         113.000000
max      514470.000000
Name: votes, dtype: float64

In [116]:
# votes가 10개 이상인 데이터만 추출
df_proposals = df_proposals[df_proposals['votes'] > 9]  # 25%
print("Over 10 votes:", df_proposals.shape, df_proposals["space_id"].nunique())
df_proposals['votes'].describe()

Over 10 votes: (21305, 33) 250


count     21305.000000
mean       1454.189392
std       17393.670192
min          10.000000
25%          24.000000
50%          59.000000
75%         186.000000
max      514470.000000
Name: votes, dtype: float64

In [ ]:
df_proposals.to_excel('proposals.xlsx', index=False, engine='xlsxwriter')

### Remove unnecessary letters

In [135]:
df_proposals = pd.read_excel('proposals.xlsx')
df_proposals.shape

(21305, 34)

In [137]:
import re

def _regex(pattern, text, output=""):
    return re.compile(pattern, re.IGNORECASE).sub(output, text)

def remove_pattern(text):
    if isinstance(text, str):
        for p in [
            r'!\[.*?\]\(.*?\)',                     # image link & alt
            r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", # email
            r"@\w+",                                # @username
            r'http[s]?://\S+|www\.\S+|ipfs://\S+',  # url and ipfs
            r'\b0x\w+\b',                           # wallet address
            r'\(\s*\)', r'\[\s*\]',                 # empty parentheses
            r'\{\s*\}', r'\<\s*\>',

        ]:
            if text:
                text = _regex(p, text)
            else:
                break
    return text

def change_pattern(text):
    if isinstance(text, str):
        for p in [
            (r'\n+', "\n"),  # multiple enter -> once
            (r'#+', "#"),    # multiple # -> once
            (r'\*+', "*"),   # multiple * -> once
        ]:
            if text:
                text = _regex(p[0], text, p[1])
            else:
                break
    return text

for v in ["title", "body"]:
    df_proposals[v] = df_proposals[v] \
        .apply(remove_pattern) \
        .apply(change_pattern) 

df_proposals = df_proposals.dropna(subset=['title', 'body'])
df_proposals.shape, df_proposals["space_id"].nunique()

((21268, 34), 250)

In [138]:
# body를 언어별로 분류
from langdetect import detect

def detect_lang(x):
    try:
        return detect(str(x))
    except:
        return None

df_proposals["lang"] = df_proposals['body'].apply(detect_lang)
df_proposals["lang"].value_counts()

en       20920
ja         175
es         105
sw           7
pt           7
de           6
zh-cn        6
ca           5
tr           5
vi           5
fr           2
it           2
sv           1
id           1
so           1
no           1
ar           1
nl           1
et           1
Name: lang, dtype: int64

In [139]:
df_proposals.to_excel('proposals_lang.xlsx', index=False, engine='xlsxwriter')

In [2]:
# 영어만 추출
df_proposals = df_proposals[df_proposals["lang"] == "en"]

In [12]:
# 유효한 문자만 추출 ~ 이모티콘 삭제
valid_pattern = re.compile(r'[a-zA-Z0-9!@#$%^&*()_\-+{}\[\]:;≥"\'<>,.?/\\|`~\s]')
def get_valid(text):
    if isinstance(text, str): 
        return ''.join(valid_pattern.findall(text))
    return text  

for v in ["title", "body"]:
    df_proposals[v] = df_proposals[v] \
        .apply(get_valid) 

In [13]:
df_proposals = df_proposals.dropna(subset=['title', 'body'])
df_proposals.shape, df_proposals["space_id"].nunique()

((20918, 34), 245)

In [14]:
df_proposals.to_excel('proposals_preprocess.xlsx', index=False, engine='xlsxwriter')

In [15]:
# df_temp = pd.read_excel('proposals_top20_0730_space_id.xlsx')
# df_temp = df_temp[["id", "Class A", "Class B", "Class C"]]
# df = pd.merge(df_proposals, df_temp, on="id", how="left")
# df.to_excel('proposals_preprocess_0812.xlsx', index=False, engine='xlsxwriter')

# TOP20

In [86]:
import pandas as pd
df_proposals = pd.read_excel('proposals_preprocess.xlsx')

In [113]:
df_proposals_top20 = df_proposals[df_proposals["space_id"].isin(TOP20)]
df_proposals_top20.shape, df_proposals_top20["space_id"].nunique()

((5366, 33), 17)

In [114]:
df_proposals_top20.to_excel('proposals_preprocess_top20.xlsx', index=False, engine='xlsxwriter')

# Vote

In [3]:
import pandas as pd

df_proposals = pd.read_excel('proposals_preprocess_0812_cleaned.xlsx')
df_proposals.shape

(20750, 39)

In [11]:
sample1 = df_proposals["space_id"].value_counts()[df_proposals["space_id"].value_counts() > 100].index
# sample2 = df_proposals["space_id"].value_counts()[df_proposals["space_id"].value_counts() <= 100].index

# with open("sample2.txt", "w", encoding="utf8") as f:
#     for i in sample2:
#         f.write(str(i) + "\n")

df = df_proposals[df_proposals["space_id"].isin(sample1)]

In [19]:
proposals = df["id"].tolist()
result = []
import time


for i, proposal in enumerate(proposals):
    print(i)
    idx = 0
    while True:
        query = dao.fetch_votes(proposal, idx)
        response = request(query).json()
        if 'error' in response.keys():
            print(proposal, response)
            time.sleep(1)
            continue
        response = response["data"]["votes"]
        if response:
            result += response
            print(len(response), len(result))
            idx += 1000
        else:
            break

0
63 63
1
689 752
2
10 762
3
60 822
4
1000 1822
1000 2822
1000 3822
1000 4822
1000 5822
1000 6822
5
59 6881
6
263 7144
7
71 7215
8
46 7261
9
51 7312
10
53 7365
11
72 7437
12
28 7465
13
43 7508
14
25 7533
15
41 7574
16
82 7656
17
46 7702
18
53 7755
19
37 7792
20
58 7850
21
571 8421
22
179 8600
23
480 9080
24
285 9365
25
32 9397
26
78 9475
27
70 9545
28
47 9592
29
45 9637
30
47 9684
31
45 9729
32
51 9780
33
91 9871
34
80 9951
35
49 10000
36
10 10010
37
73 10083
38
36 10119
39
26 10145
40
63 10208
41
1000 11208
1000 12208
184 12392
42
1000 13392
1000 14392
271 14663
43
109 14772
44
101 14873
45
80 14953
46
45 14998
47
74 15072
48
10 15082
49
299 15381
50
517 15898
51
155 16053
QmNhYY3oSNiZN7fGNFD6Xe94kgYoZRUPFfNNwTttsdoAm5 {'error': 'unauthorized', 'error_description': 'too many requests, refer to https://docs.snapshot.org/tools/api/api-keys#limits'}
QmNhYY3oSNiZN7fGNFD6Xe94kgYoZRUPFfNNwTttsdoAm5 {'error': 'unauthorized', 'error_description': 'too many requests, refer to https://docs.snap

In [21]:
import pickle

for i in range(len(result)):
    result[i]["proposal_id"] = result[i]["proposal"]["id"]
    result[i]["space_id"] = result[i]["space"]["id"]
    del result[i]["proposal"]
    del result[i]["space"]

df_votes = pd.DataFrame(dict(
    id = [x["id"] for x in result],
    vp = [x["vp"] for x in result],
    app = [x["app"] for x in result],
    ipfs = [x["ipfs"] for x in result],
    voter = [x["voter"] for x in result],
    choice = [x["choice"] for x in result],
    reason = [x["reason"] for x in result],
    created = [x["created"] for x in result],
    metadata = [x["metadata"] for x in result],
    vp_state = [x["vp_state"] for x in result],
    vp_by_strategy = [x["vp_by_strategy"] for x in result],
    proposal_id = [x["proposal_id"] for x in result],
    space_id = [x["space_id"] for x in result],  
))


with open('parrot.pkl', 'wb') as f:
    pickle.dump(result, f)
    

In [25]:
groups = df_votes["proposal_id"].value_counts()

count = 0
failed = []
omg = []

for i, row in df.iterrows():
    if row["id"] == 'QmRjmmFqy6str8zsj53CNEhZJRZUWFeMtGMwb3vEwKzDNn':
        continue
    if groups[row["id"]] != row["votes"]:
        print(row["id"], groups[row["id"]], row["votes"], row["space_id"])
        count += 1
        if row["votes"] <= 12000:
            failed.append(row["id"])
        else:
            omg.append(row["id"])
            
print(count)
len(failed)

0x1751d8de3c549ee99fbc9c1286d9575c482c3e639500dcc027455c8742d48bc9 6000 14455 aave.eth
0xedb87c640a24dadde80f39db547d8ea913483b638efb045fc9ac1a4132d7b78f 6000 6250 aave.eth
0xc31254fac1369090cea7c0105cbc6381b72189c038391996f855708ff2e0c02e 6000 7857 aave.eth
0xda3519f11e2308239d5f179b2579fe921b2a421eb752aba959779ee9ecea0d69 6000 9998 aave.eth
0x2cbfeb5c5329c4d1720dbae4045386fceb2d2a9bd5b42a81cee47ea5c934f491 6000 9607 aave.eth
0xb4c756ac701b5cbc0ec2d832652e4f1c820aa40f7d9be18eb81f87080a52c5f1 6000 10119 aave.eth
0x4f97f52ef593bbdddc1037597729e1575ed8d4df997c1781334795b569ca25ce 6000 10911 aave.eth
0x84deca82139320b2570f04211b249e37b8a7602b4a0ed70e6fa772c9f6e94550 6000 18710 aave.eth
0xf6bc25809a931910ea490b86b3c847dc30e09fc0d278ebf0bcdd48f4868e35c1 6000 16363 aave.eth
0x2c71c8f2e17fd2f2cc403750c65bfba464b8af45ee6c53c40961c8dad4987c8a 6000 16152 aave.eth
0xbda28d65ca4d64005e6019948ed52d9d62c9e73e356ab1013aa2d4829f40c735 6000 10786 aave.eth
0x833eca942053ffcaecbedd6c3b0d5f2392f1ad868e57f

124

In [26]:
with open("omg.txt", "w") as f:
    for item in omg:
        f.write("%s\n" % item)

In [28]:
result_failed = []


for i, proposal in enumerate(failed+omg):
    print(i)
    idx = 0
    while True:
        query = dao2.fetch_votes(proposal, idx, False)
        response = request(query).json()
        if 'error' in response.keys():
            print(proposal, response)
            time.sleep(1)
            continue
        response = response["data"]["votes"]
        if response:
            result_failed += response
            print(len(response), len(result_failed))
            idx += 1000
        else:
            break

0
1000 1000
1000 2000
1000 3000
1000 4000
1000 5000
1000 6000
1
1000 7000
1000 8000
1000 9000
1000 10000
1000 11000
1000 12000
2
1000 13000
1000 14000
1000 15000
1000 16000
1000 17000
1000 18000
3
1000 19000
1000 20000
1000 21000
1000 22000
1000 23000
1000 24000
4
1000 25000
1000 26000
1000 27000
1000 28000
1000 29000
1000 30000
5
1000 31000
1000 32000
1000 33000
1000 34000
1000 35000
1000 36000
6
1000 37000
1000 38000
1000 39000
1000 40000
1000 41000
1000 42000
7
1000 43000
1000 44000
1000 45000
1000 46000
1000 47000
1000 48000
8
1000 49000
1000 50000
1000 51000
1000 52000
1000 53000
1000 54000
9
1000 55000
1000 56000
1000 57000
1000 58000
1000 59000
1000 60000
10
1000 61000
1000 62000
1000 63000
1000 64000
1000 65000
1000 66000
11
1000 67000
1000 68000
1000 69000
1000 70000
1000 71000
1000 72000
12
1000 73000
1000 74000
1000 75000
1000 76000
1000 77000
1000 78000
13
1000 79000
1000 80000
1000 81000
1000 82000
1000 83000
1000 84000
14
1000 85000
1000 86000
1000 87000
1000 88000
1000 8

In [30]:
for i in range(len(result_failed)):
    result_failed[i]["proposal_id"] = result_failed[i]["proposal"]["id"]
    result_failed[i]["space_id"] = result_failed[i]["space"]["id"]
    del result_failed[i]["proposal"]
    del result_failed[i]["space"]

df_votes_failed = pd.DataFrame(dict(
    id = [x["id"] for x in result_failed],
    vp = [x["vp"] for x in result_failed],
    app = [x["app"] for x in result_failed],
    ipfs = [x["ipfs"] for x in result_failed],
    voter = [x["voter"] for x in result_failed],
    choice = [x["choice"] for x in result_failed],
    reason = [x["reason"] for x in result_failed],
    created = [x["created"] for x in result_failed],
    metadata = [x["metadata"] for x in result_failed],
    vp_state = [x["vp_state"] for x in result_failed],
    vp_by_strategy = [x["vp_by_strategy"] for x in result_failed],
    proposal_id = [x["proposal_id"] for x in result_failed],
    space_id = [x["space_id"] for x in result_failed],  
))

with open('parrot_failed.pkl', 'wb') as f:
    pickle.dump(result_failed, f)

In [31]:
df_v = pd.concat([df_votes, df_votes_failed]).drop_duplicates(subset=["id"])
df_v.shape

(8222094, 13)

In [41]:
groups = df_v.groupby("space_id")
for name, group in groups:
    try:
        if group.shape[0] < 1048576:
            group.to_excel(f'space_vote/votes_{name}.xlsx', index=False)
            continue
        with pd.ExcelWriter(f'space_vote/votes_{name}.xlsx', engine='xlsxwriter') as writer:
            for i in range(0, group.shape[0], 1048570):
                group[i:i+1048570].to_excel(writer, sheet_name=str(i), index=False)
        
    except Exception as e:
        print(name, group.shape)
        print(e)

In [39]:
df_v[["space_id", "proposal_id"]].value_counts().to_frame().rename(columns={0:"count"}).sort_index().to_excel('describe.xlsx')

In [36]:
groups = df_v["proposal_id"].value_counts()


not_found = [
    # 'QmRjmmFqy6str8zsj53CNEhZJRZUWFeMtGMwb3vEwKzDNn',
]

none = dict(
    space_id = [],
    proposal_id = [],
    proposal_vote_count = [],
    real_vote_count = [],
)

for i, row in df.iterrows():
    if row["id"] in not_found:
        none["space_id"].append(row["space_id"])
        none["proposal_id"].append(row["id"])
        none["proposal_vote_count"].append(row["votes"])
        none["real_vote_count"].append(0)
        continue
    if groups[row["id"]] != row["votes"]:
        none["space_id"].append(row["space_id"])
        none["proposal_id"].append(row["id"])
        none["proposal_vote_count"].append(row["votes"])
        none["real_vote_count"].append(groups[row["id"]])

df_none = pd.DataFrame(none)
df_none.head()

,space_id,proposal_id,proposal_vote_count,real_vote_count
0,aave.eth,0x1751d8de3c549ee99fbc9c1286d9575c482c3e639500...,14455,12000
1,aave.eth,0x84deca82139320b2570f04211b249e37b8a7602b4a0e...,18710,12000
2,aave.eth,0xf6bc25809a931910ea490b86b3c847dc30e09fc0d278...,16363,12000
3,aave.eth,0x2c71c8f2e17fd2f2cc403750c65bfba464b8af45ee6c...,16152,12000
4,aave.eth,0x833eca942053ffcaecbedd6c3b0d5f2392f1ad868e57...,16995,12000


In [37]:
# df_none에 multi-index 적용

df_none = df_none.set_index(["space_id", "proposal_id"])
df_none.to_excel("mismatch.xlsx")